In [2]:
from utils import db_info

import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import *

#### 데이터베이스 접속
- data.ini를 사용해 hide variable 진행

In [4]:
connection_format = f'postgresql://{db_info["USER"]}:{db_info["PASSWORD"]}@{db_info["HOST"]}:{db_info["PORT"]}/{db_info["DB"]}'
        
client = create_engine(connection_format)

### Cluster_1
- 모든 환자에 대해 총 내원일수를 구하고 총 내원일수의 최대값과 총 내원일수 최대값을 가지는 환자수 출력
    - 분석함수를 사용해 총 내원일수들의 최대값을 구함 : max_visit_period
    - 내원한 환자 중 내원일수(visit_period)와 최대 내원일수(max_visit_period)가 같은 환자들의 count 추출
    
#### RESULT
- 내원한 환자들의 최대 내원일수는 3731일 이며, 3731일간 내원한 환자 수는 1명이다.

In [5]:
QUERY_1 = """
SELECT max_visit_period, count(*) OVER () AS patient_count
FROM (
    SELECT (visit_end_date - visit_start_date + 1) AS visit_period
         , MAX (visit_end_date - visit_start_date + 1) OVER () AS max_visit_period
    FROM visit_occurrence
) AS A
WHERE visit_period = max_visit_period;
"""

In [6]:
cluster_1 = pd.read_sql(QUERY_1, con=client)
cluster_1

,max_visit_period,patient_count
0,3731,1


### Cluster_2
- 환자들이 진단 받은 상병 내역 중 첫글자는 (a, b, c, d, e)로 시작하고 중간에 'heart'가 포함된 상병 이름 출력
    - 조건
        1. 문자 검색시 대소문자 구분 X
        2. 상병 이름을 중복없이 나열
    - 과정
        - 두 테이블을 JOIN하여 id에 대한 name 정보를 확보
        - 상병 이름(concept_name) 중 첫자리가 a, b, c, d, e(`^[a-e]`) 이고, 중간에 heart(`*heart*`)가 들어간 행만 추출
        - 중복 제거를 위해 DISTINCT 예약어 사용
        
#### RESULT
- 조건을 모두 만족하는 상병은 `Chronic congestive heart failure` 만 존재한다.

In [7]:
QUERY_2 = """
SELECT DISTINCT B.concept_name
FROM condition_occurrence A
JOIN concept B
ON A.condition_concept_id = B.concept_id
WHERE B.concept_name ~* '^[a-e].*heart*';
"""

In [8]:
cluster_2 = pd.read_sql(QUERY_2, con=client)
cluster_2

,concept_name
0,Chronic congestive heart failure


### Cluster_3
- 특정 환자(1891866)에게 처방된 약의 종류별로 처음 시작일, 마지막 종료일, 복용일에 대한 정보 추출
    - 조건
        1. 복용일(term) : 마지막 종료일과 처음시작일의 차이 + 1
        2. 복용일이 긴 순으로 정렬
    - 과정
        - 특정 환자에 대한 데이터만 추출한 뒤, 약 종류별로 그룹핑을 진행(group by)
        - 약 종류별로 처음 시작일 중 최소값(해당 약을 최초로 복용한 날짜 : start_date), 마지막 종료일 중 최대값(해당 약을 최후로 복용한 날짜 : end_date)에 대한 컬럼 생성
        - 위 과정을 진행한 쿼리를 인라인 뷰에 넣고, end_date와 start_date 간의 차이(term)를 구하고 term을 기준으로 내림차순 진행
        
#### RESULT
- 해당 환자는 19009384, 19030765, 40213154, 1539463, 40213227 에 대한 약을 복용하였다.
- 각 약에 대한 복용일은 14190, 3640, 3221, 2921, 1일 이다.

In [23]:
QUERY_3 = """
SELECT drug_concept_id, start_date, end_date, (end_date - start_date + 1) AS term
FROM (
    SELECT
          drug_concept_id
        , MIN(drug_exposure_start_date) AS start_date
        , MAX(drug_exposure_end_date) AS end_date
    FROM drug_exposure
    WHERE person_id = 1891866
    GROUP BY drug_concept_id
     ) AS A
ORDER BY term DESC;
"""

In [24]:
cluster_3 = pd.read_sql(QUERY_3, con=client)
cluster_3

,drug_concept_id,start_date,end_date,term
0,19009384,1959-12-01,1998-10-06,14190
1,19030765,1988-10-18,1998-10-05,3640
2,40213154,1989-09-12,1998-07-07,3221
3,1539463,1990-03-13,1998-03-11,2921
4,40213227,1993-01-05,1993-01-05,1


### Cluster_4
- 15가지의 약의 처방 건수와 각 약별로 짝지어진 약의 처방 건수 중 두번째 약의 처방 건수가 첫번째 약의 처방 건수보다 더 많은 첫번째 약품명을 출력
    - 조건
        1. 처방건수 순으로 출력 : 두번째 약의 처방 건수 순으로 진행함
        2. 명시된 VIEW테이블 활용
            a. drugs : drug_concept_id(첫번째약 번호), concept_name(약품명)
            b. prescription_count : drug_concept_id(첫번째약 번호), cnt(처방건수)
            c. drug_pair : drug_concept_id1(첫번째약 번호), drug_concept_id2(두번째약 번호)
            
    - 과정
        - FROM절 인라인뷰(d_pc) : drugs와 prescription_count을 JOIN하여 첫번째 약들에 대한 처방 건수 출력(id1_cnt)
        - JOIN절 인라인뷰(dp_pc) : drug_pair와 prescription_count을 JOIN하여 짝지어진 두번째 약들에 대한 처방 건수 출력(id2_cnt)
        - 두 테이블의 첫번째 약에 대한 번호를 기준으로 JOIN한 뒤, id1_cnt보다 id2_cnt가 큰 약들만 추출
        - filtering된 약들 중 id2_cnt에 대해 내림차순한 약 이름(concept_name) 출력
        
#### RESULT
- 첫번째 약 처방 건수보다 짝지어진 두번째 약 처방 건수가 많은 약들은 총 10개가 존재한다.

In [25]:
QUERY_4 = """
WITH drug_list AS (
SELECT DISTINCT drug_concept_id, concept_name, count(*) AS cnt
FROM drug_exposure
JOIN concept
ON drug_concept_id = concept_id
WHERE concept_id IN (40213154, 19078106, 19009384, 40224172, 19127663, 1511248, 40169216, 1539463
                    , 19126352, 1539411, 1332419, 40163924, 19030765, 19106768, 19075601)
GROUP BY drug_concept_id, concept_name
ORDER BY count(*) DESC 
)
, drugs AS (
SELECT drug_concept_id, concept_name
FROM drug_list
)
, prescription_count AS (
SELECT drug_concept_id, cnt
FROM drug_list
)

SELECT concept_name
FROM (
    SELECT d.drug_concept_id, d.concept_name, pc.cnt AS id1_cnt
    FROM drugs d
    JOIN prescription_count pc
    ON d.drug_concept_id = pc.drug_concept_id
) AS d_pc
JOIN (
    SELECT drug_pair.*, pc.cnt AS id2_cnt
    FROM drug_pair
    JOIN prescription_count pc
    ON drug_pair.drug_concept_id2 = pc.drug_concept_id
) AS dp_pc
ON d_pc.drug_concept_id = dp_pc.drug_concept_id1
WHERE id1_cnt < id2_cnt
ORDER BY id2_cnt DESC;
"""

In [26]:
cluster_4 = pd.read_sql(QUERY_4, con=client)
cluster_4

,concept_name
0,hydrochlorothiazide 25 MG Oral Tablet
1,amlodipine 5 MG / hydrochlorothiazide 12.5 MG ...
2,hydrochlorothiazide 12.5 MG Oral Tablet
3,24 HR metformin hydrochloride 500 MG Extended ...
4,atenolol 50 MG / chlorthalidone 25 MG Oral Tab...
5,1 ML epoetin alfa 4000 UNT/ML Injection [Epogen]
6,120 ACTUAT fluticasone propionate 0.044 MG/ACT...
7,simvastatin 20 MG Oral Tablet
8,amlodipine 5 MG Oral Tablet
9,clopidogrel 75 MG Oral Tablet


### Cluster_5
- 조건에 해당하는 환자수 추출
    - 조건
        1. 제 2형 당뇨병을 진단받은 환자
        2. 18세 이상 환자
        3. 진단 이후 Metformin을 90일 이상 복용한 환자
    - 과정
        - diabetes_list : 조건 1에 대한 VIEW 테이블 생성
            - Age를 구하기 위해 TO_CHAR를 통해 condition_start_date에 대한 year 데이터만 추출한 컬럼 생성 : start_condition_year
        - over_90 : 조건 3에 대한 VIEW 테이블 생성
            - 처방에 대한 데이터셋에서 처방 종료 일시(drug_exposure_end_date)와 처방 시작 일시(drug_exposure_start_date) 간의 차이에 대한 컬럼 생성 : exposure_term
        - FROM 인라인뷰(diabetes_over_18) : diabetes_list, person 테이블을 person_id로 JOIN한 뒤, start_condition_year를 INT로 형반환한 값과 year_of_birth의 차이가 18이상인 행만 추출
        - 90일 이상 복용한 환자(over_90)와 diabetes_over_18를 person_id로 JOIN한 뒤, 복용일(exposure_term)이 90이상인 환자만 추출
        
#### RESULT
- 제 2형 당뇨병을 진단받고, Metformin을 90일 이상 복용한 18세 이상을 환자들의 수는 30이다.

In [27]:
QUERY_5 = """
WITH diabetes_list AS (
SELECT *, TO_CHAR(condition_start_date, 'YYYY') AS start_condition_year
FROM condition_occurrence
WHERE condition_concept_id IN (3191208, 36684827, 3194332, 3193274, 43531010, 4130162
                              ,45766052, 45757474, 4099651, 4129519, 4063043, 4230254
                              ,4193704, 4304377, 201826, 3194082, 3192767)
)
, over_90 AS (
SELECT *, (drug_exposure_end_date - drug_exposure_start_date + 1) AS exposure_term
FROM drug_exposure
WHERE drug_concept_id = 40163924
)

SELECT COUNT(DISTINCT over_90.person_id) AS cnt -- drug_exposure에서 특정 환자마다 해당 약품처방을 여러번의 진단에서 동일하게 받은 경우도 있기에
FROM (
    SELECT dl.person_id
    FROM diabetes_list dl
    JOIN person p
    ON dl.person_id = p.person_id
    WHERE (CAST(dl.start_condition_year AS INT) - p.year_of_birth) >= 18
) AS diabetes_over_18
JOIN over_90
ON diabetes_over_18.person_id = over_90.person_id
WHERE over_90.exposure_term >= 90;
"""

In [28]:
cluster_5 = pd.read_sql(QUERY_5, con=client)
cluster_5

,cnt
0,30


### Cluster_6
- 제 2형 당뇨병을 진단받은 환자에서 추출한 환자군의 의약품 처방이 변경된 패턴을 추출
    - 조건
        1. 제 2형 당뇨병을 진단받은 환자
        2. 5가지의 drug_concept_id만 사용
        2. 빈도에 대해 내림차순으로 정렬
    - 과정
        - diabetes_list : 조건 1에 대한 VIEW 테이블 생성
        - drug_cluster : 조건 2에 대한 VIEW 테이블 생성
            - 스칼라 뷰로 조건 1을 만족하는 환자에 대해서만 추출 진행
        - drug_cluster 테이블에서 person_id, drug_concept_id로 그룹핑 진행
        - 그룹별 최소 복용 시간(min_date)를 구하고, person_id로 파티션을 구분하고 min_date기준으로 정렬
        - 분석함수 LAG와 LEAD를 사용해 각각 이전 처방의 약(lag_drug_concept_id) 이후 처방의 약(lead_min_date)에 대한 정보 추출
        
#### RESULT
- 해당 과정을 통해 추출한 데이터셋을 Python으로 추가적인 전처리 진행

In [15]:
QUERY_6 = """
WITH diabetes_list AS (
SELECT *
FROM condition_occurrence
WHERE condition_concept_id IN (3191208, 36684827, 3194332, 3193274, 43531010, 4130162
                              ,45766052, 45757474, 4099651, 4129519, 4063043, 4230254
                              ,4193704,4304377,201826,3194082,3192767)
)
, drug_cluster AS (
SELECT *
FROM (
    SELECT *
    FROM drug_exposure
    WHERE drug_concept_id IN (19018935, 1539411, 1539463, 19075601, 1115171)
) AS de
WHERE person_id IN (SELECT person_id FROM diabetes_list)
)

SELECT person_id, drug_concept_id, MIN(drug_exposure_start_date) AS min_date
     , LAG (drug_concept_id, 1) OVER (PARTITION BY person_id ORDER BY MIN(drug_exposure_start_date)) AS lag_drug_concept_id
     , LEAD (MIN(drug_exposure_start_date), 1) OVER (PARTITION BY person_id ORDER BY MIN(drug_exposure_start_date)) AS lead_min_date
FROM drug_cluster
GROUP BY person_id, drug_concept_id 
ORDER BY person_id, min_date;
"""

#### SQL을 사용한 전처리

In [18]:
cluster_6 = pd.read_sql(QUERY_6, con=client)
cluster_6.head()

,person_id,drug_concept_id,min_date,lag_drug_concept_id,lead_min_date
0,31196,1539463,1997-05-14,NaN,2014-08-26
1,31196,1115171,2014-08-26,1539463.0,None
2,50663,1115171,2014-08-01,NaN,None
3,67212,1115171,2016-11-19,NaN,None
4,170280,1115171,2002-09-30,NaN,None


### Python을 사용한 전처리

- pettern_list : 약이 총 5가지 이므로 `a~e`로 정의
- person_list : 추출한 환자들의 고유값

In [19]:
pettern_list = ['a', 'b', 'c', 'd', 'e']
person_list = cluster_6['person_id'].unique().tolist()

#### 각 환자별 패턴을 정의하는 함수 생성

In [20]:
def defind_pettern(df, person_id):
    # 인자로 들어온 person_id에 대한 데이터만 추출
    personal_df = df[df['person_id'] == person_id]
    
    personal_pettern = []
    cnt = 0
    while True:
        # 현재 행의 drug_concept_id가 lag_drug_concept_id의 값과 다르다면 (약을 처방하는 패턴이 바뀜)
        if personal_df.iloc[cnt]['lag_drug_concept_id'] != personal_df.iloc[cnt]['drug_concept_id']:
            # 현재 행의 min_date가 lead_min_date과 다르다면 (동시에 처방하지 않음)
            if personal_df.iloc[cnt]['min_date'] != personal_df.iloc[cnt]['lead_min_date']:
                # pettern_list 중 첫번째 패턴을 넣고 다음 패턴으로 변경
                personal_pettern.append(pettern_list[cnt])
                cnt += 1
            # 현재 행의 min_date가 lead_min_date과 같다면 (동시에 처방)
            else:
                # 2개의 패턴 추출
                p1, p2 = pettern_list[cnt], pettern_list[cnt+1]
                # 패턴에 대한 포맷 정의
                pettern = f'({p1},{p2})'
                # 패턴 저장
                personal_pettern.append(pettern)
                # 두 개의 행에 대해 진행했기에 cnt + 2
                cnt += 2
                # cnt가 해당 환자의 처방건수 이상이면 종료
                if cnt >= len(personal_df):
                    break
                # 아직 처방건수가 존재한다면
                else:
                    # 다음 행의 drug_concept_id가 이전의 drug_concept_id과 같다면 두번째 패턴을 저장
                    if personal_df.iloc[cnt]['lag_drug_concept_id'] == personal_df.iloc[cnt]['drug_concept_id']:
                        personal_pettern.append(p2)
                    # 다음 행의 drug_concept_id가 두 개 이전의 drug_concept_id과 같다면 첫번째 패턴을 저장
                    elif personal_df.iloc[cnt]['drug_concept_id'] == personal_df.iloc[cnt-2]['drug_concept_id']:
                        personal_pettern.append(p1)
        
        # cnt가 해당 환자의 처방건수 이상이면 종료
        if cnt >= len(personal_df):
            break
            
    return personal_pettern

#### 각 환자별 패턴 정의 진행
- 패턴 사이를 `,`에서 `->`로 변경

In [21]:
pattern_df = pd.DataFrame(columns=['pattern'])
result = []
for person_id in person_list:
    pattern = defind_pettern(cluster_6, person_id)
    pattern = ' -> '.join(pattern)
    
    pattern_df.loc[len(pattern_df)] = pattern
    
pattern_df.head()

,pattern
0,a -> b
1,a
2,a
3,a
4,"a -> (b,c) -> d"


#### 패턴별 수에 대한 데이터프레임 생성

In [22]:
pattern_df = pattern_df.groupby('pattern').size().reset_index(name='person_count')
pattern_df = pattern_df.sort_values('person_count', ascending=False).reset_index(drop=True)
pattern_df

,pattern,person_count
0,a,12
1,a -> b,10
2,a -> b -> c,3
3,"(a,b)",2
4,"(a,b) -> c",1
5,"a -> (b,c) -> d",1
